In [157]:
import os
import pandas as pd
import datetime as dt
import time, pytz
import calendar
import numpy as np
from statistics import mean

from uce_resources import get_site_id, get_mms_data, get_applied_forecast, get_prices, get_green_tariff, get_applied_forecast_old
from uce_resources import make_results

In [158]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

c:\Users\o.babenko\.conda\envs\uce_daily\lib\site-packages\ipykernel_launcher.py:7: SAWarning: Did not recognize type 'point' of column 'location'
  import sys


In [179]:
from settings.sites import ceg as sites_list

target_year = 2023
target_month = 3
forecasts_types = ['real', 'naive', 'zero', 'raw']

target_folder = 'C:/Users/o.babenko/Downloads/Verkhivka/'
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

sites_list =['Verkhivka']

sites_data = dict.fromkeys(sites_list)
print(sites_data)

{'Verkhivka': None}


In [180]:
# from sqlalchemy import create_engine, MetaData
# from sqlalchemy.pool import NullPool
# from settings.db import DO_URL

# engine = create_engine(DO_URL, poolclass=NullPool)
# metadata = MetaData()
# metadata.reflect(bind=engine)

In [181]:
with engine.connect() as connection:
    prices = get_prices(target_year, target_month, connection, metadata.tables['electricity_market_prices'], currency='UAH')

min_price_day = (prices.index.min() + dt.timedelta(days=1)).day
max_price_day = prices.index.max().day

start_date = dt.date(year=target_year, month=target_month, day=min_price_day)
end_date = dt.date(year=target_year, month=target_month, day=max_price_day)
print(start_date, end_date)

prices.to_excel(target_folder + 'prices_{}_{}_{}-{}.xlsx'.format(target_year, target_month, min_price_day, max_price_day))
prices

2023-03-01 2023-03-31


,dam,imsp,positive_unbalance,negative_unbalance
2023-02-28 22:30:00,1.99800,0.26998,0.25648,2.09790
2023-02-28 23:30:00,1.99800,0.12766,0.12128,2.09790
2023-03-01 00:30:00,1.99800,0.00002,0.00002,2.09790
2023-03-01 01:30:00,1.99800,0.00002,0.00002,2.09790
2023-03-01 02:30:00,1.99800,0.00002,0.00002,2.09790
...,...,...,...,...
2023-03-31 16:30:00,3.90000,0.00001,0.00001,4.09500
2023-03-31 17:30:00,3.90000,4.67998,3.70500,4.91398
2023-03-31 18:30:00,3.98265,4.77332,3.78352,5.01199
2023-03-31 19:30:00,3.90000,4.67998,3.70500,4.91398


In [182]:
limitations_file = './data/limitations/limitations.xlsx'

limitation_data = pd.read_excel(
    limitations_file, 
    sheet_name='{}_{:02d}'.format(target_year, target_month),
    parse_dates=False
    )

limitation_data = limitation_data.fillna(0)
limitation_data.iloc[:, 2:] = limitation_data.iloc[:, 2:].astype(int)
limitation_data['True_Time'] = (limitation_data['Time'] - 1).apply(lambda x: '{:02d}'.format(x))
limitation_data.iloc[:, :2] = limitation_data.iloc[:, :2].astype(str)

print(limitation_data.loc[limitation_data['Date'] == '2022-03-27'])

index = pd.to_datetime(limitation_data['Date'] + ' ' + limitation_data['True_Time'], format='%Y-%m-%d %H') + dt.timedelta(minutes=30)
index = pd.DatetimeIndex(data=index)
index = index.tz_localize(pytz.timezone('europe/kiev'))
index = index.tz_convert('utc')
index = index.tz_localize(None)
limitation_data.index = index
limitation_data = limitation_data.drop(columns=['Date', 'Time', 'True_Time'])

extended_limitations = pd.DataFrame(index=prices.index, columns=limitation_data.columns, data=0)
extended_limitations.loc[extended_limitations.index.intersection(limitation_data.index)] = limitation_data.loc[limitation_data.index.intersection(extended_limitations.index)]
limitation_data = extended_limitations
limitat=pd.concat([limitation_data['Verkhivka']], axis=1, join="outer")
print(limitat)


with pd.ExcelWriter(target_folder + 'lim_{}_{}_{}_UAH.xlsx'.format(str(sites_list[0]), target_year, target_month), engine="openpyxl") as  writer:
    limitat.to_excel(writer, 'limitat')
# limitation_data.to_excel("limitations123.xlsx")

Empty DataFrame
Columns: [Date, Time, Afanasiivka, Bazaltova, Balivka, Balky, Bar, Bereznehuvate, Bilashky, Bilozerka, Vasylivka, Velihen, Veliton, Verkhivka, Hlybochok_1, Hlybochok_2.1, Hlybochok_2.2, Hnatkiv, Dibrovka, Yelanets_1, Yelanets_2, Inhulets_1, Inhulets_2, Kachkarivka, Kyselivka, Komyshany_1, Komyshany_2, Kopaihorod, Kostohryzove, Kulevcha, Liubymivka, Mala_Lepetykha, Mykolaivka, Myroliubivka, Novokondakove, Oleshky_1, Oleshky_2, Pohrebyshche, Poniativka, Porohy, Rubanivka, Solone, Stanislavchyk, Stepnohirsk, Teplychna, Cherniatka, Chechelnyk_1, Chechelnyk_2, Sharhorod_1, Shyroke, True_Time]
Index: []

[0 rows x 51 columns]
                     Verkhivka
2023-02-28 22:30:00          0
2023-02-28 23:30:00          0
2023-03-01 00:30:00          0
2023-03-01 01:30:00          0
2023-03-01 02:30:00          0
...                        ...
2023-03-31 16:30:00          0
2023-03-31 17:30:00          0
2023-03-31 18:30:00          0
2023-03-31 19:30:00          0
2023-03-31 20:3